## Movielens clustering

In this Notebook, we are looking for user clusters in the Movielens data, using _k_-means clustering.

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans #The k-means algorithm

First, let's start by creating a user-item matrix, as explained in the other Notebook.

In [2]:
movie_file = pd.read_csv('movies.csv')
ratings_file = pd.read_csv('ratings.csv')
df = pd.merge(movie_file, ratings_file)

ratings = pd.pivot_table(df, index='userId', columns='title', values='rating')
ratings.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's put the most popular movies at the front.

In [3]:
#This piece of code is a bit complex. Here it is, step by step:
#1. reindex shuffles a dataframe according to a new list
#2. ratings.count() gets the number of non-NaN values per column/movie
#3. sort_values() sort those values, descending (because ascending=False)
#4. finally, .index gets the names of the columns/movies
#axis=1 tells Pandas we want to reshuffle the columns (not the rows)
ratings = ratings.reindex(ratings.count().sort_values(ascending=False).index, axis=1)
ratings.head(3)

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,"Last Wedding, The (Kivenpyörittäjän kylä) (1995)","Last Winter, The (2006)",Last Year's Snow Was Falling (1983),Last of the Dogmen (1995),Late Marriage (Hatuna Meuheret) (2001),Late Night Shopping (2001),Late Night with Conan O'Brien: The Best of Triumph the Insult Comic Dog (2004),"Late Shift, The (1996)",Latter Days (2003),'71 (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,3.0,4.0,5.0,5.0,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will now find clusters. Unfortunately, the _k_-means algorithm won't work with NaN values. We will put a 0 in the empty cells. This is not ideal for many reasons, but the best we can do for now without getting really complex

In [4]:
ratings_full = ratings.fillna(0) #fill the NaN with the mean of each column
ratings_full.head(3)

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,"Last Wedding, The (Kivenpyörittäjän kylä) (1995)","Last Winter, The (2006)",Last Year's Snow Was Falling (1983),Last of the Dogmen (1995),Late Marriage (Hatuna Meuheret) (2001),Late Night Shopping (2001),Late Night with Conan O'Brien: The Best of Triumph the Insult Comic Dog (2004),"Late Shift, The (1996)",Latter Days (2003),'71 (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,3.0,4.0,5.0,5.0,4.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finish the code below. You need to...
1. Pick a suitable number of clusters (somewhere between 4 and 10 will work)
2. Apply the k-means algorithm to the Movielens user-item matrix that is in the code. Store the cluster predictions in the original `ratings` dataframe and continue working with that dataframe.
3. Print the number of users per cluster (do you remember the relevant Pandas function?).
4. Calculate the mean rating by user cluster using the Pandas pivot_table function. Pandas will sort alphabetically after making the pivot table, so you will need to reorder your pivot table with `my_pivot.reindex(ratings.count().sort_values(ascending=False).index, axis=1)`. Replace `my_pivot` with the name of your pivot table.
5. Examine the mean ratings of the top rated movies by user cluster. Can you describe the user clusters in plain language (e.g., ‘simple-minded action movie lover’)? This may be hard…


In [18]:
km = KMeans(n_clusters=5) #create a new k-means model with 3 clusters
X = ratings_full.iloc[:9719] #get the X variables from the dataframe
km = km.fit(X) #calculate the cluster centers
ratings_full['cluster'] = km.predict(X) #predict the clusters of each observation and store in the dataframe
ratings_full.head()

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,"Last Winter, The (2006)",Last Year's Snow Was Falling (1983),Last of the Dogmen (1995),Late Marriage (Hatuna Meuheret) (2001),Late Night Shopping (2001),Late Night with Conan O'Brien: The Best of Triumph the Insult Comic Dog (2004),"Late Shift, The (1996)",Latter Days (2003),'71 (2014),cluster
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,3.0,4.0,5.0,5.0,4.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,0.0,0.0,1.0,5.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5,0.0,3.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [19]:
ratings_full['cluster'].value_counts()

4    367
2     98
3     95
1     45
0      5
Name: cluster, dtype: int64

In [20]:
mean_clusters = pd.pivot_table(data=ratings_full, index='cluster')
mean_clusters

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
cluster,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.200000,0.6,1.300000,0.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.100000,0.600000,0.000000,1.700000,0.000000
1,0.088889,0.000000,0.000000,0.000000,0.000000,0.033333,0.322222,0.0,1.388889,0.288889,...,0.033333,0.355556,0.144444,0.133333,0.000000,0.533333,0.822222,0.133333,0.444444,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.042105,0.036842,0.000000,0.000000,0.000000,0.221053,0.0,0.115789,0.000000,...,0.000000,0.089474,0.000000,0.000000,0.000000,0.310526,0.184211,0.042105,0.410526,0.010526
4,0.000000,0.000000,0.009537,0.013624,0.021798,0.000000,0.034060,0.0,0.201635,0.019074,...,0.000000,0.021798,0.012262,0.000000,0.008174,0.043597,0.024523,0.000000,0.038147,0.000000


In [21]:
mean_clusters.reindex(ratings.count().sort_values(ascending=False).index, axis=1)

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,Shrooms (2007),Siam Sunset (1999),Side by Side (2012),Sightseers (2012),"Signal, The (2007)",Shot Caller (2017),"Signal, The (2014)",Silent Hill: Revelation 3D (2012),Silent Movie (1976),'71 (2014)
cluster,,,,,,,,,,,,,,,,,,,,,
0,3.700000,3.500000,4.800000,4.200000,4.900000,4.500000,3.000000,3.200000,3.400000,3.400000,...,0.000000,0.000000,0.6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4.055556,3.677778,3.855556,3.377778,4.088889,4.022222,3.344444,2.622222,3.133333,2.577778,...,0.077778,0.000000,0.0,0.1,0.077778,0.000000,0.000000,0.066667,0.077778,0.088889
2,3.290816,3.173469,3.193878,2.744898,0.132653,0.443878,2.811224,3.224490,2.326531,1.943878,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.384211,3.126316,3.189474,3.000000,3.484211,3.710526,2.621053,2.447368,2.678947,2.600000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.430518,1.670300,1.294278,1.182561,1.670300,1.260218,0.551771,0.743869,0.678474,0.978202,...,0.000000,0.013624,0.0,0.0,0.000000,0.009537,0.012262,0.000000,0.000000,0.000000


We see that cluster 4 is a hard rater which rates a lot of movies really low. We can see that cluster 3 shows that fiction is not a really big like.(Star wars/matrix) It is hard to see which genre is with which cluster because you have to know the moviegenres. 